In [46]:
# import shutil

# file_name = "open.zip"
# # output_dir = "dataset/dataset"
# output_dir = "."

# format = "zip"
# shutil.unpack_archive(file_name, output_dir, format)

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

SEED = 777

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [48]:
df_train = pd.read_csv('train.csv')
print(df_train.head(2))

        img_id                 img_path  airplane  airport  bare soil  \
0  TRAIN_00000  ./train/TRAIN_00000.jpg         0        0          1   
1  TRAIN_00001  ./train/TRAIN_00001.jpg         0        0          1   

   baseball diamond  basketball court  beach  bridge  buildings  ...  tanks  \
0                 0                 0      0       0          0  ...      0   
1                 0                 0      0       0          1  ...      0   

   tennis court  terrace  track  trail  transmission tower  trees  water  \
0             0        0      0      0                   0      1      0   
1             0        0      0      0                   0      0      0   

   wetland  wind turbine  
0        0             0  
1        0             0  

[2 rows x 62 columns]


In [49]:
df_test = pd.read_csv('test.csv')
print(df_test.head(2))

       img_id               img_path
0  TEST_00000  ./test/TEST_00000.jpg
1  TEST_00001  ./test/TEST_00001.jpg


In [50]:
print(len(df_train), len(df_test)) # 65496, 43665

65496 43665


In [51]:
# df_train.info()

In [52]:
# !pip install iterative-stratification

In [53]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms # , models
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# 이미지 경로와 레이블을 나눕니다
image_paths = df_train['img_path'].values
labels = df_train.iloc[:, 2:].values  # 레이블 열 선택

In [54]:
# 사용자 정의 데이터셋 클래스
class SatelliteDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx].astype(np.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

In [55]:
# 이미지 전처리 및 데이터 증강
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Larger size for more complex models
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [56]:
k_folds = 5
skf = MultilabelStratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

In [57]:
from torchvision.models import resnet50

num_classes = labels.shape[1]
print(num_classes)

60


In [58]:
from torch.optim import lr_scheduler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [59]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, fold, num_epochs=10):
    best_val_loss = float('inf')
    best_epoch = -1
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)

        if epoch >= 9:
            torch.save(model.state_dict(), f'{fold}_epoch{epoch}.pt')
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        scheduler.step()

        if epoch % 3 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model = model.state_dict()
    print(fold, 'fold | best_val_loss :', best_val_loss, ', best_epoch :', best_epoch)
    return best_model

In [60]:
best_models = []
for fold, (train_idx, val_idx) in tqdm(enumerate(skf.split(image_paths, labels))):
    print(f'Fold {fold + 1}/{k_folds}')
    
    train_paths = image_paths[train_idx]
    val_paths = image_paths[val_idx]
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]
    
    train_dataset = SatelliteDataset(train_paths, train_labels, transform=train_transform)
    val_dataset = SatelliteDataset(val_paths, val_labels, transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    model = resnet50(pretrained=True)
    
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes),
        nn.Sigmoid()
    )
    
    model = model.to(device)
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=50)
    
    best_model = train_model(model, train_loader, val_loader, criterion, optimizer, fold, num_epochs=50)
    
    best_models.append(best_model)

0it [00:00, ?it/s]

Fold 1/5


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Train Loss: 0.1657, Val Loss: 0.1111
Epoch 4/50, Train Loss: 0.0991, Val Loss: 0.0780
Epoch 7/50, Train Loss: 0.0861, Val Loss: 0.0710
Epoch 10/50, Train Loss: 0.0778, Val Loss: 0.0668
Epoch 13/50, Train Loss: 0.0728, Val Loss: 0.0644
Epoch 16/50, Train Loss: 0.0683, Val Loss: 0.0598
Epoch 19/50, Train Loss: 0.0648, Val Loss: 0.0597
Epoch 22/50, Train Loss: 0.0619, Val Loss: 0.0573
Epoch 25/50, Train Loss: 0.0590, Val Loss: 0.0572
Epoch 28/50, Train Loss: 0.0561, Val Loss: 0.0542
Epoch 31/50, Train Loss: 0.0542, Val Loss: 0.0538
Epoch 34/50, Train Loss: 0.0533, Val Loss: 0.0546
Epoch 37/50, Train Loss: 0.0512, Val Loss: 0.0564
Epoch 40/50, Train Loss: 0.0502, Val Loss: 0.0539
Epoch 43/50, Train Loss: 0.0477, Val Loss: 0.0547
Epoch 46/50, Train Loss: 0.0463, Val Loss: 0.0557
Epoch 49/50, Train Loss: 0.0448, Val Loss: 0.0539


1it [1:22:10, 4930.04s/it]

...

2 fold | best_val_loss : 0.051895222650090875 , best_epoch : 36
Fold 4/5
Epoch 1/50, Train Loss: 0.1684, Val Loss: 0.1161
Epoch 4/50, Train Loss: 0.0999, Val Loss: 0.0808
Epoch 7/50, Train Loss: 0.0854, Val Loss: 0.0764
Epoch 10/50, Train Loss: 0.0779, Val Loss: 0.0685
Epoch 13/50, Train Loss: 0.0721, Val Loss: 0.0680
Epoch 16/50, Train Loss: 0.0687, Val Loss: 0.0641
Epoch 19/50, Train Loss: 0.0641, Val Loss: 0.0634
Epoch 22/50, Train Loss: 0.0611, Val Loss: 0.0637
Epoch 25/50, Train Loss: 0.0584, Val Loss: 0.0619
Epoch 28/50, Train Loss: 0.0563, Val Loss: 0.0609
Epoch 31/50, Train Loss: 0.0538, Val Loss: 0.0594
Epoch 34/50, Train Loss: 0.0519, Val Loss: 0.0605
Epoch 37/50, Train Loss: 0.0503, Val Loss: 0.0610
Epoch 40/50, Train Loss: 0.0494, Val Loss: 0.0574
Epoch 43/50, Train Loss: 0.0475, Val Loss: 0.0615
Epoch 46/50, Train Loss: 0.0461, Val Loss: 0.0581
Epoch 49/50, Train Loss: 0.0449, Val Loss: 0.0572


4it [5:28:37, 4925.01s/it]

3 fold | best_val_loss : 0.05721002376025043 , best_epoch : 48
Fold 5/5
Epoch 1/50, Train Loss: 0.1685, Val Loss: 0.1204
Epoch 4/50, Train Loss: 0.0991, Val Loss: 0.0822
Epoch 7/50, Train Loss: 0.0847, Val Loss: 0.0715
Epoch 10/50, Train Loss: 0.0777, Val Loss: 0.0653
Epoch 13/50, Train Loss: 0.0715, Val Loss: 0.0710
Epoch 16/50, Train Loss: 0.0674, Val Loss: 0.0752
Epoch 19/50, Train Loss: 0.0640, Val Loss: 0.0634
Epoch 22/50, Train Loss: 0.0608, Val Loss: 0.0595
Epoch 25/50, Train Loss: 0.0589, Val Loss: 0.0570
Epoch 28/50, Train Loss: 0.0571, Val Loss: 0.0583
Epoch 31/50, Train Loss: 0.0545, Val Loss: 0.0594
Epoch 34/50, Train Loss: 0.0530, Val Loss: 0.0604
Epoch 37/50, Train Loss: 0.0510, Val Loss: 0.0607
Epoch 40/50, Train Loss: 0.0495, Val Loss: 0.0642
Epoch 43/50, Train Loss: 0.0476, Val Loss: 0.0581
Epoch 46/50, Train Loss: 0.0471, Val Loss: 0.0633
Epoch 49/50, Train Loss: 0.0455, Val Loss: 0.0659


5it [6:50:39, 4927.90s/it]

4 fold | best_val_loss : 0.0563434624211723 , best_epoch : 43


In [61]:
len(best_models)

5

In [62]:
# 모델 평가 함수
def evaluate_model(model, val_loader):
    model.eval()
    val_loss = 0.0
    corrects = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            
            preds = outputs > 0.5
            corrects += (preds == labels).sum().item()
    
    val_loss /= len(val_loader.dataset)
    
    accuracy = corrects / (len(val_loader.dataset) * labels.size(1))
    
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.4f}')

In [63]:
for model_state in best_models:
    model.load_state_dict(model_state)
    evaluate_model(model, val_loader)

Validation Loss: 0.0343, Validation Accuracy: 0.9856
Validation Loss: 0.0365, Validation Accuracy: 0.9846
Validation Loss: 0.0321, Validation Accuracy: 0.9862
Validation Loss: 0.0355, Validation Accuracy: 0.9851
Validation Loss: 0.0355, Validation Accuracy: 0.9851


In [64]:
class TestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_path

In [65]:
test_dataset = TestDataset(df_test['img_path'].values, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [66]:
ensemble_predictions = np.zeros((len(df_test), num_classes))
k_folds = len(best_models)
for model_state in best_models:
    model = resnet50(pretrained=True)
    
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes),
        nn.Sigmoid()
    )
    
    model.load_state_dict(model_state)
    model = model.to(device)
    
    model.eval()
    fold_predictions = []
    with torch.no_grad():
        for images, img_paths in test_loader:
            images = images.to(device)
            outputs = model(images)
            fold_predictions.append(outputs.cpu().numpy())
    
    fold_predictions = np.concatenate(fold_predictions, axis=0)
    ensemble_predictions += fold_predictions

# Average the predictions
ensemble_predictions /= k_folds

In [67]:
submission = pd.DataFrame(ensemble_predictions, columns=df_train.columns[2:])
submission.insert(0, 'img_id', df_test['img_id'])
submission.to_csv('submission.csv', index=False)

print("Test predictions saved to submission.csv")

Test predictions saved to submission.csv
